# Red tipo mapa de Kohonen
---
Implementación de red tipo mapa de Kohonen para procesar caracteres escritos a mano para reconocer caracteres (clasificación con LVQ) y asociar caracteres parciales con caracteres completos (memoria asociativa).

## Análisis del dataset a utilizar
El dataset provisto por la cátedra es un archivo formato CSV, una tabla, donde cada fila es una lista con el valor de los pixeles de una imágen de un caracter. Estas imágenes son de 28 x 28 pixeles, es decir, la lista es de 784 números de 0 a 255.

EL dataset brindado da las imágenes con etiqueta, por lo que se posee información de qué caracter corresponde a cada imágen.

In [2]:
import csv
import numpy as np

In [ ]:
train_raw = []
with open('train.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for i, row in enumerate(csvReader):
        if (i == 0):
            continue
        label = row[0]
        train_raw.append([label, np.array(row[1:], dtype=float)])
        if (i > 10):
            break

La cantidad de imágenes en el conjunto de entrenamiento es la siguiente:

In [ ]:
with open('train.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    row_count = sum(1 for row in csvReader)
print(row_count-1)

## Visualización previa de los datos

Para poder observar los datos ingresados previo a implementar el algoritmo se reconstruyen una muestra de 5 imágenes del dataset:

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
collection = []

number = np.ndarray((28, 28))

for img in train_raw:
    iter = 0
    for i in range(0, 28):
        for j in range(0, 28):
            number[i, j] = (img[1][iter])
            iter += 1
    number = np.zeros((28, 28))
    collection.append([img[0], number])

In [ ]:
f, axes = plt.subplots(5, 1, figsize=(5, 5))
for i, ax in enumerate(axes):
  ax.imshow(collection[i][1])
  print(collection[i+1][0]) # Ni idea porque los labels están corridos

### Normalización de los datos de entrada

Con el fin de preparar los datos de entrada al algoritmo, se normalizan los valores de la imágen, pasando de 0 a 255 a valores entre 0 y 1.

In [ ]:
from numpy import delete

train = []

for i in range(0, len(train_raw)-1):
    # label = img[0]
    label = train_raw[i+1][0]
    delete(train_raw[i][0], 0)
    img_as_float = train_raw[i][1] / 255
    train.append([label, img_as_float])

In [ ]:
print(train[0][0], train[0][1].shape)

## Modelo de la Red Neuronal

In [ ]:
x = train_raw[0][1]

map_kohonen = np.zeros((100, 100)) # Mapa de neuronas

W = np.zeros((map_kohonen.shape[0], map_kohonen.shape[1], x.shape[0]))        

In [ ]:
print(W.shape, W[0, 0, :].shape)

### Ritmo de aprendizaje $\alpha(t)$
Se seleccionó como ritmo de aprendizaje un función decreciente exponencial de la forma:

$\alpha(t)=\alpha_{0}\left(\frac{\alpha_{f}}{\alpha_{0}}\right)^{\frac{t}{t_{\alpha}}}$

In [ ]:
def ritmo_aprendizaje(t, t_alfa=100, alfa_0=0.7, alfa_f=0.01):
    alfa = alfa_0 * (alfa_f / alfa_0)**(t / t_alfa)
    return alfa

Ejemplo de la función:

In [ ]:
a = []
for i in range(0, 150):
    a.append(ritmo_aprendizaje(i))

plt.plot(range(0, 150), a)
plt.grid()

### Función de vecindad $h(|\bar{i}-\bar{g}|, t)$
Para comenzar, se eligió una función simple como es la escalón, donde el radio de vecindad R(t) desciende linealmente.

In [ ]:
def radio_vecindad(t, t_r=100, R_0=100, R_f=1):
    R = R_0 + (R_f - R_0) * t / t_r
    return R

In [ ]:
def vecindad(i, g, t, R=radio_vecindad):
    aux = np.sqrt((i[0] - g[0])**2 + (i[1] - g[1])**2)
    if (aux > R(t)):
        return 0
    else:
        return 1

Gráfica de la función utilizada como radio de vecindad:

In [ ]:
a = []
for i in range(0, 100):
    a.append(radio_vecindad(i))

plt.plot(range(0, 100), a)
plt.grid()

### Métricas de similitud
Se seleccionó como métrica de similitud la distancia euclídea:

$d^{2}(\bar{w_{ij}}, \bar{x}) = \displaystyle\sum_{k=1}^{n} (w_{ij} - x_{k})^{2}$

In [ ]:
def d_euclidean(x, w):
    sum = 0
    for i in range(x.shape[0]):
        sum += (x[i] - w[i])**2
    return sum

Solo para calcular la similitud de un $w_{ijk}$ con una entrada $x$, se debe recorrer todo el vector, es decir, k iteraciones (en este caso 785 iteraciones).

### Reglas de aprendizaje para la métrica de similitud usada
La regla de aprendizaje para la distancia euclídea es de la forma:

$\bar{w_{i}}(t+1) = \bar{w_{i}}(t) + \alpha(t) \cdot h(|\bar{i} - \bar{g}|, t) \cdot (x_{k}(t) - w_{ijk}(t))$

In [ ]:
def update_w(w, x, i, g, t, lvq, ritmo_aprendizaje, vecindad):
    w_new = w + lvq * ritmo_aprendizaje(t) * vecindad(i, g, t) * (x - w) 
    # CUIDADO: Ver si la diferencia puede dar negativo, 
    # si es así, calcular valor absoluto. Se utiliza distancia euclideana,
    # por lo que no debería pasar.
    return w_new

## Análisis de las diferentes etapas de entrenamiento de la red neuronal

### Inicialización de pesos sinápticos
Se inicializan los pesos sinápticos con números aleatorios pequeños tomados de una distribución Gaussiana con $\mu=0$ y $\sigma=1$:

In [ ]:
from numpy.random import normal

mu = 0
sigma = 1

for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        W[i, j, :] = abs(0.01 * normal(mu, sigma, W.shape[2]))

### Obtención de neurona ganadora

La neurona ganadora $\bar{g}$ cumple:

$\bar{g} = \text{arg}\quad\displaystyle\text{min}_{ij}\left[ d(\bar{w_{ij}}, \bar{x})\right]$

In [ ]:
from math import inf

minim = inf

for i in range(map_kohonen.shape[0]):
    for j in range(map_kohonen.shape[1]):
        map_kohonen[i, j] = d_euclidean(x, W[i, j, :])
        if (map_kohonen[i, j] < minim):
            minim = map_kohonen[i, j]
            g = [i, j]

In [ ]:
g

### Actualización de pesos sinápticos

In [ ]:
t = 0
for [_, x] in train:
    minim = inf

    for i in range(map_kohonen.shape[0]):
        for j in range(map_kohonen.shape[1]):
            map_kohonen[i, j] = d_euclidean(x, W[i, j, :])
            if (map_kohonen[i, j] < minim):
                minim = map_kohonen[i, j]
                g = [i, j]
    
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W[i, j, :] = update_w(W[i, j, :], x, [i, j], g, t, 1, ritmo_aprendizaje, vecindad)
    
    t += 1

## Evaluación de aprendizaje y optimización de hiperparámetros

Para la evaluación del aprendizaje de la red se realizó **k-fold cross validation**, con $k=3$, es decir, se deja $1/3$ de los casos de entrenamiento para testing y el experimento se repite $3$ veces.

In [5]:
data_raw = []
with open('train.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for i, row in enumerate(csvReader):
        if (i == 0):
            continue
        label = row[0]
        data_raw.append([label, np.array(row[1:], dtype=float)])

Normalización de todo el dataset:

In [6]:
data = []

for i in range(0, len(data_raw)-1):
    # label = img[0]
    label = data_raw[i+1][0]
    delete(data_raw[i][0], 0)
    img_as_float = data_raw[i][1] / 255
    data.append([label, img_as_float])

NameError: name 'delete' is not defined

Siendo la longitud del dataset:

In [ ]:
len(data)

Se procede a separar un tercio del dataset que formará el conjunto de test:

In [ ]:
train = data[:28000]

test = data[28000:]

print(len(data), len(train), len(test))

## Entrenamiento de la red neuronal

Dentro del conjunto de training, se destina un 10% a un ajuste fino con **LVQ (Learning Vector Quantization)**, recordar que el problema es de **clasificación**.

In [ ]:
t = 0
for [_, x] in train[:25200]:
    minim = inf

    for i in range(map_kohonen.shape[0]):
        for j in range(map_kohonen.shape[1]):
            map_kohonen[i, j] = d_euclidean(x, W[i, j, :])
            if (map_kohonen[i, j] < minim):
                minim = map_kohonen[i, j]
                g = [i, j]
    
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W[i, j, :] = update_w(W[i, j, :], x, [i, j], g, t, 1, ritmo_aprendizaje, vecindad)
    
    t += 1